In [ ]:
#keras的应用模块提供了预训练的keras模型权重
#可以用来预测 特征提取 参数调优
#模型的参数会下载到 类unix: ~/.keras/models/ 
#windows: %USERPROFILE%/.keras/models/

In [1]:
#使用keras的预训练参数 示例
#resnet50
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input , decode_predictions
import numpy as np


model = ResNet50(weights='imagenet')

img_path = 'dog.jpg'
img = image.load_img(img_path , target_size=(224,224))

x = image.img_to_array(img)
x = np.expand_dims(x , axis=0)
x = preprocess_input(x)

preds = model.predict(x)

print('prediction' , decode_predictions(preds , top=3)[0])

In [2]:
#vgg16 特征提取
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np


base_model = VGG16(weights='imagenet' , include_top=False)
#构建中间模型
model = Model(inputs = base_model.input , outputs=base_model.get_layer('block4_pool').output)

img_path = 'cat.jpg'
img = image.load_img(img_path , target_size=(224,224))

x=image.img_to_array(img)
x=np.expand_dims(x , axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)

In [2]:
#inceptionv3 新的数据上参数调优（迁移学习） 冷冻一些层
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense , GlobalAveragePooling2D
from keras import backend as K


base_model = InceptionV3(weights='imagenet' , include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024 , activation='relu')(x)

predictions = Dense(200 , activation='softmax')(x)

model = Model(inputs=base_model.input , outputs=predictions)

#将原始inceptionv3中的层全部冻结起来
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop' , loss='categorical_crossentropy')
model.fit() #在自己的数据上训练
#model.fit_generator()
#局部调优 对一些层参数进行调优 冻结一些层 训练剩下的层
for i , layer in enumerate(base_model.layers):
    print(i , layer.name)
#将前249层冻结 不训练
for layer in model.layers[:249]:
    layer.trainable = False
#训练后面的层
for layer in model.layers[249:]:
    layer.trainable = True
#重新compile层 重新训练
model.compile(optimizer='sgd',loss='categorical_crossentropy')

model.fit() #model.fit_generator()

In [4]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

#自定制的tensor
input_tensor = Input(shape=(224,224,3))

model.InceptionV3(input_tensor=input_tensor ,weights='imagenet',include_top=True)

In [3]:
from keras.applications.xception import Xception

#默认图片size 299*299
Xception(include_top=True , weights='imagenet',
        input_tensor = None , input_shape=None,
        pooling = None , classes=1000)

In [ ]:
#xception
#vgg16
#vgg19
#resnet50
#inceptionv3
#inceptionv4
#inceptionresnetv2
#mobilenet